In [2]:
!python --version

Python 3.7.5


# Web Scraping from sofifa.com

We will use the following libraries:
- `requests` to send HTTP requests to sofifa.com and get web pages programmatically.
- `BeautifulSoup` for parsing web pages and extracting information.
- `pandas` to create a tabular structure of the data we extract and save it locally.

In [3]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd

print('Library versions:\n')
print(f'requests: {requests.__version__}')
print(f'BeautifulSoup: {bs4.__version__}')
print(f'pandas: {pd.__version__}')

Library versions:

requests: 2.22.0
BeautifulSoup: 4.8.2
pandas: 0.25.3


By default, sofifa.com displays only 15 attributes. We can check all the attributes on their site and see what the URL becomes. We will use that URL to programmatically fetch all the attributes of the players:

In [4]:
BASE_URL='https://sofifa.com/players?showCol%5B0%5D=pi&showCol%5B1%5D=ae&showCol%5B2%5D=hi&showCol%5B3%5D=wi&showCol%5B4%5D=pf&showCol%5B5%5D=oa&showCol%5B6%5D=pt&showCol%5B7%5D=bo&showCol%5B8%5D=bp&showCol%5B9%5D=gu&showCol%5B10%5D=jt&showCol%5B11%5D=le&showCol%5B12%5D=vl&showCol%5B13%5D=wg&showCol%5B14%5D=rc&showCol%5B15%5D=ta&showCol%5B16%5D=cr&showCol%5B17%5D=fi&showCol%5B18%5D=he&showCol%5B19%5D=sh&showCol%5B20%5D=vo&showCol%5B21%5D=ts&showCol%5B22%5D=dr&showCol%5B23%5D=cu&showCol%5B24%5D=fr&showCol%5B25%5D=lo&showCol%5B26%5D=bl&showCol%5B27%5D=to&showCol%5B28%5D=ac&showCol%5B29%5D=sp&showCol%5B30%5D=ag&showCol%5B31%5D=re&showCol%5B32%5D=ba&showCol%5B33%5D=tp&showCol%5B34%5D=so&showCol%5B35%5D=ju&showCol%5B36%5D=st&showCol%5B37%5D=sr&showCol%5B38%5D=ln&showCol%5B39%5D=te&showCol%5B40%5D=ar&showCol%5B41%5D=in&showCol%5B42%5D=po&showCol%5B43%5D=vi&showCol%5B44%5D=pe&showCol%5B45%5D=cm&showCol%5B46%5D=td&showCol%5B47%5D=ma&showCol%5B48%5D=sa&showCol%5B49%5D=sl&showCol%5B50%5D=tg&showCol%5B51%5D=gd&showCol%5B52%5D=gh&showCol%5B53%5D=gk&showCol%5B54%5D=gp&showCol%5B55%5D=gr&showCol%5B56%5D=tt&showCol%5B57%5D=bs&showCol%5B58%5D=wk&showCol%5B59%5D=sk&showCol%5B60%5D=aw&showCol%5B61%5D=dw&showCol%5B62%5D=ir&showCol%5B63%5D=pac&showCol%5B64%5D=sho&showCol%5B65%5D=pas&showCol%5B66%5D=dri&showCol%5B67%5D=def&showCol%5B68%5D=phy'

Also, attributes of only 60 players are displayed at a time. We can decide which 60 players we view using the `offset` flag in the URL.

For example, `offset=0` will fetch the details of the first 60 players (player 1 - player 60), `offset=60` will fetch details of players 61 - 120, and so on. So we will keep increasing the offset by 60 and fetching a new list of players until we've got details of all the players.

In [5]:
# This method gets details of 60 players from the specified offset
def get_players_from_offset(offset=0):
    
    # Create an array that will hold all the information to be returned
    result = []
    
    # Form the URL
    url = f'{BASE_URL}&offset={offset}'
    
    # Make the HTTP call to get the page
    page_source = requests.get(url)
    
    # Parse the HTML page using BeautifulSoup
    soup = BeautifulSoup(page_source.text, 'html.parser')
    
    # Get the table that contains all the player attributes
    table_body = soup.find('tbody')
    
    # Each row in the table is a different player.
    # Loop over each and extract the attributes.
    for row in table_body.findAll('tr'):
        
        # The `extract_information` method defined below extracts
        # the attributes of a single player from a table row
        info = extract_information(row)
        result.append(info)
        
    return result
        
    
# Extract the information of a single player from an HTML table row
def extract_information(row):
    
    player = {}
    
    td = row.findAll('td')
    
    player['id'] = td[6].text
    player['name'] = td[1].findAll('a')[1].text
    player['age'] = td[2].text
    player['height'] = td[7].text
    player['weight'] = td[8].text
    player['preferred_foot'] = td[9].text
    player['overall_rating'] = td[3].text
    player['potential'] = td[4].text
    player['team'] = td[5].find('a').text
    player['best_overall'] = td[10].text
    player['best_position'] = td[11].text
    player['growth'] = td[12].text
    player['joined'] = td[13].text
    player['loan_date_end'] = td[14].text
    player['value'] = td[15].text
    player['wage'] = td[16].text
    player['release_clause'] = td[17].text
    
    player['total_attacking'] = td[18].text
    player['crossing'] = td[19].text
    player['finishing'] = td[20].text
    player['heading_accuracy'] = td[21].text
    player['short_passing'] = td[22].text
    player['volleys'] = td[23].text
    
    player['total_skill'] = td[24].text
    player['dribbling'] = td[25].text
    player['curve'] = td[26].text
    player['fk_accuracy'] = td[27].text
    player['long_passing'] = td[28].text
    player['ball_control'] = td[29].text
    
    player['total_movement'] = td[30].text
    player['acceleration'] = td[31].text
    player['sprint_speed'] = td[32].text
    player['agility'] = td[33].text
    player['reactions'] = td[34].text
    player['balance'] = td[35].text
    
    player['total_power'] = td[36].text
    player['shot_power'] = td[37].text
    player['jumping'] = td[38].text
    player['stamina'] = td[39].text
    player['strength'] = td[40].text
    player['long_shots'] = td[41].text
    
    player['mentality'] = td[42].text
    player['aggression'] = td[43].text
    player['interception'] = td[44].text
    player['positioning'] = td[45].text
    player['vision'] = td[46].text
    player['penalties'] = td[47].text
    player['composure'] = td[48].text
    
    player['total_defending'] = td[49].text
    player['marking'] = td[50].text
    player['standing_tackle'] = td[51].text
    player['sliding_tackle'] = td[52].text
    
    player['total_goalkeeping'] = td[53].text
    player['gk_diving'] = td[54].text
    player['gk_handling'] = td[55].text
    player['gk_kicking'] = td[56].text
    player['gk_positioning'] = td[57].text
    player['gk_reflexes'] = td[58].text
    
    player['total_stats'] = td[59].text
    player['base_stats'] = td[60].text
    
    player['weak_foot'] = td[61].text.strip()
    player['skill_moves'] = td[62].text.strip()
    player['attacking_work_rate'] = td[63].text
    player['defensive_work_rate'] = td[64].text
    
    player['international_reputation'] = td[65].text.strip()
    
    player['PAC'] = td[66].text
    player['SHO'] = td[67].text
    player['PAS'] = td[68].text
    player['DRI'] = td[69].text
    player['DEF'] = td[70].text
    player['PHY'] = td[71].text
    
    return player

In [6]:
data = get_players_from_offset()
print(data[0])

{'id': '239053', 'name': 'F. Valverde', 'age': '20', 'height': '6\'0"', 'weight': '172lbs', 'preferred_foot': 'Right', 'overall_rating': '81', 'potential': '89', 'team': 'Real Madrid', 'best_overall': '82', 'best_position': 'CM', 'growth': '8', 'joined': 'Jul 22, 2016', 'loan_date_end': 'N/A', 'value': '€26M', 'wage': '€115K', 'release_clause': '€58.5M', 'total_attacking': '339', 'crossing': '63', 'finishing': '69', 'heading_accuracy': '59', 'short_passing': '82', 'volleys': '66', 'total_skill': '376', 'dribbling': '77', 'curve': '70', 'fk_accuracy': '66', 'long_passing': '83', 'ball_control': '80', 'total_movement': '370', 'acceleration': '74', 'sprint_speed': '77', 'agility': '69', 'reactions': '83', 'balance': '67', 'total_power': '378', 'shot_power': '80', 'jumping': '54', 'stamina': '85', 'strength': '79', 'long_shots': '80', 'mentality': '369', 'aggression': '77', 'interception': '79', 'positioning': '74', 'vision': '80', 'penalties': '59', 'composure': '80', 'total_defending': '

In [49]:
player_data = []
for i in range(0, 21000, 60):
    players = get_players_from_offset( i )
    player_data = player_data + players
    print(f'{i + 60} players fetched...')

60 players fetched...
120 players fetched...
180 players fetched...
240 players fetched...
300 players fetched...
360 players fetched...
420 players fetched...
480 players fetched...
540 players fetched...
600 players fetched...
660 players fetched...
720 players fetched...
780 players fetched...
840 players fetched...
900 players fetched...
960 players fetched...
1020 players fetched...
1080 players fetched...
1140 players fetched...
1200 players fetched...
1260 players fetched...
1320 players fetched...
1380 players fetched...
1440 players fetched...
1500 players fetched...
1560 players fetched...
1620 players fetched...
1680 players fetched...
1740 players fetched...
1800 players fetched...
1860 players fetched...
1920 players fetched...
1980 players fetched...
2040 players fetched...
2100 players fetched...
2160 players fetched...
2220 players fetched...
2280 players fetched...
2340 players fetched...
2400 players fetched...
2460 players fetched...
2520 players fetched...
2580 play

In [50]:
df = pd.DataFrame(player_data)
df.head()

,id,name,age,height,weight,preferred_foot,overall_rating,potential,team,best_overall,...,skill_moves,attacking_work_rate,defensive_work_rate,international_reputation,PAC,SHO,PAS,DRI,DEF,PHY
0,239053,F. Valverde,20,"6'0""",172lbs,Right,81,89,Real Madrid,82,...,3,High,High,1,76,73,77,77,73,79
1,225591,L. Suárez,23,"5'6""",132lbs,Left,71,79,Villarreal CF,73,...,4,Medium,Low,1,65,68,73,76,23,35
2,238160,M. Demiral,21,"6'4""",187lbs,Right,74,85,Juventus,75,...,2,Medium,High,1,68,39,48,57,75,75
3,236499,Douglas Luiz,21,"5'10""",154lbs,Right,74,87,Aston Villa,74,...,3,High,Medium,1,67,67,72,74,69,70
4,212198,Bruno Fernandes,24,"5'10""",152lbs,Right,85,88,Sporting CP,86,...,4,High,High,2,77,83,87,85,66,74


In [51]:
df.shape

(21000, 71)

In [53]:
df = df.drop_duplicates(subset=['id'])

In [54]:
df.shape

(19570, 71)

In [55]:
df.to_csv('../data/raw_data.csv')